In [1]:
# TASK 1: DATA COLLECTION
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
print("TASK 1: THU THẬP VÀ KIỂM TRA DỮ LIỆU")


TASK 1: THU THẬP VÀ KIỂM TRA DỮ LIỆU


In [2]:
# 1.1. LOAD DATASET
print("\n[1/4] Loading MovieLens 25M dataset...")
movies = pd.read_csv('../data/raw/movies.csv')
ratings = pd.read_csv('../data/raw/ratings.csv')
tags = pd.read_csv('../data/raw/tags.csv')
links = pd.read_csv('../data/raw/links.csv')
print("Load thành công!")


[1/4] Loading MovieLens 25M dataset...
Load thành công!


In [3]:
# 1.2. KIỂM TRA YÊU CẦU ĐỀ BÀI
print("\n[2/4] Kiểm tra yêu cầu đề bài...")

print(f"\n📌 Yêu cầu 1: Số items ≥ 2,000")
print(f"   → Số movies: {len(movies):,}")

print(f"\n📌 Yêu cầu 2: Có ≥ 5 features")
print(f"   → Features hiện tại: {movies.columns.tolist()}")
print(f"   → Số features: {len(movies.columns)}")


[2/4] Kiểm tra yêu cầu đề bài...

📌 Yêu cầu 1: Số items ≥ 2,000
   → Số movies: 62,423

📌 Yêu cầu 2: Có ≥ 5 features
   → Features hiện tại: ['movieId', 'title', 'genres']
   → Số features: 3


In [4]:
# 1.3. THỐNG KÊ DATASET
print("\n[3/4] Thống kê tổng quan...")

print(f"\nThống kê cơ bản:")
print(f"   • Tổng số movies:  {len(movies):,}")
print(f"   • Tổng số ratings: {len(ratings):,}")
print(f"   • Tổng số users:   {ratings['userId'].nunique():,}")
print(f"   • Tổng số tags:    {len(tags):,}")

print(f"\nRatings distribution:")
print(ratings['rating'].value_counts().sort_index())

print(f"\nRating statistics:")
print(ratings['rating'].describe())


[3/4] Thống kê tổng quan...

Thống kê cơ bản:
   • Tổng số movies:  62,423
   • Tổng số ratings: 25,000,095
   • Tổng số users:   162,541
   • Tổng số tags:    1,093,360

Ratings distribution:
rating
0.5     393068
1.0     776815
1.5     399490
2.0    1640868
2.5    1262797
3.0    4896928
3.5    3177318
4.0    6639798
4.5    2200539
5.0    3612474
Name: count, dtype: int64

Rating statistics:
count    2.500010e+07
mean     3.533854e+00
std      1.060744e+00
min      5.000000e-01
25%      3.000000e+00
50%      3.500000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64


In [5]:
# 1.4. TẠO THÊM FEATURES (OPTIMIZED VERSION)
print("\n[4/4] Tạo thêm features để đủ yêu cầu...")

# Feature 1,2,3: movieId, title, genres (có sẵn)
print("Feature 1: movieId")
print("Feature 2: title")  
print("Feature 3: genres")

# Feature 4: Extract year từ title
if 'year' not in movies.columns:
    movies['year'] = movies['title'].str.extract(r'\((\d{4})\)', expand=False)
    movies['year'] = pd.to_numeric(movies['year'], errors='coerce')
    print("Feature 4: year")

# Feature 5,6,7: Tính rating stats
if 'rating_avg' not in movies.columns:
    rating_stats = ratings.groupby('movieId').agg({
        'rating': ['mean', 'count', 'std']
    }).reset_index()
    rating_stats.columns = ['movieId', 'rating_avg', 'rating_count', 'rating_std']
    
    movies = movies.merge(rating_stats, on='movieId', how='left')
    print("Feature 5: rating_avg")
    print("Feature 6: rating_count")  
    print("Feature 7: rating_std")

# Feature 8: Tags 
if len(tags) > 0 and 'tags_combined' not in movies.columns:
    tags_clean = tags.copy()
    tags_clean['tag'] = tags_clean['tag'].fillna('').astype(str)
    
    # Lọc empty tags trước khi groupby (giảm data size)
    tags_clean = tags_clean[tags_clean['tag'] != '']
    
    # Dùng agg trực tiếp thay vì apply lambda
    movie_tags = tags_clean.groupby('movieId')['tag'].agg(' '.join).reset_index()
    movie_tags.columns = ['movieId', 'tags_combined']
    
    movies = movies.merge(movie_tags, on='movieId', how='left')
    movies['tags_combined'] = movies['tags_combined'].fillna('')
    print("Feature 8: tags_combined (optimized)")

# Feature 9: Clean title
if 'title_clean' not in movies.columns:
    movies['title_clean'] = movies['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True).str.strip()
    print("Feature 9: title_clean")

# XÓA CÁC CỘT TRÙNG LẶP (nếu có)
duplicate_cols = [col for col in movies.columns if col.endswith('_x') or col.endswith('_y')]
if duplicate_cols:
    print(f"\nPhát hiện {len(duplicate_cols)} cột trùng lặp: {duplicate_cols}")
    
    # Xóa cột _x, giữ _y và rename
    cols_to_drop = [col for col in duplicate_cols if col.endswith('_x')]
    movies = movies.drop(columns=cols_to_drop)
    
    # Rename _y về tên gốc
    rename_dict = {col: col.replace('_y', '') for col in movies.columns if col.endswith('_y')}
    movies = movies.rename(columns=rename_dict)
    
    print("Đã xóa cột trùng lặp!")

print(f"\nTổng số features cuối cùng: {len(movies.columns)}")
print(f"Danh sách features: {movies.columns.tolist()}")


[4/4] Tạo thêm features để đủ yêu cầu...
Feature 1: movieId
Feature 2: title
Feature 3: genres
Feature 4: year
Feature 5: rating_avg
Feature 6: rating_count
Feature 7: rating_std
Feature 8: tags_combined (optimized)
Feature 9: title_clean

Tổng số features cuối cùng: 9
Danh sách features: ['movieId', 'title', 'genres', 'year', 'rating_avg', 'rating_count', 'rating_std', 'tags_combined', 'title_clean']


In [6]:
# HIỂN THỊ KẾT QUẢ
print("TỔNG KẾT")

print(f"\nTổng số features hiện tại: {len(movies.columns)}")
print(f"   Features: {movies.columns.tolist()}")

print(f"\nSample data với features mới:")
display(movies[['movieId', 'title_clean', 'year', 'genres', 'rating_avg', 'rating_count']].head(10))

print(f"\nMovies có nhiều ratings nhất:")
display(movies.nlargest(10, 'rating_count')[['title_clean', 'rating_avg', 'rating_count', 'genres']])

print(f"\nMovies có rating cao nhất (≥100 ratings):")
popular_movies = movies[movies['rating_count'] >= 100]
display(popular_movies.nlargest(10, 'rating_avg')[['title_clean', 'rating_avg', 'rating_count', 'genres']])

TỔNG KẾT

Tổng số features hiện tại: 9
   Features: ['movieId', 'title', 'genres', 'year', 'rating_avg', 'rating_count', 'rating_std', 'tags_combined', 'title_clean']

Sample data với features mới:


,movieId,title_clean,year,genres,rating_avg,rating_count
0,1,Toy Story,1995.0,Adventure|Animation|Children|Comedy|Fantasy,3.893708,57309.0
1,2,Jumanji,1995.0,Adventure|Children|Fantasy,3.251527,24228.0
2,3,Grumpier Old Men,1995.0,Comedy|Romance,3.142028,11804.0
3,4,Waiting to Exhale,1995.0,Comedy|Drama|Romance,2.853547,2523.0
4,5,Father of the Bride Part II,1995.0,Comedy,3.058434,11714.0
5,6,Heat,1995.0,Action|Crime|Thriller,3.854909,24588.0
6,7,Sabrina,1995.0,Comedy|Romance,3.363666,12132.0
7,8,Tom and Huck,1995.0,Adventure|Children,3.114583,1344.0
8,9,Sudden Death,1995.0,Action,2.992051,3711.0
9,10,GoldenEye,1995.0,Action|Adventure|Thriller,3.421458,28265.0



Movies có nhiều ratings nhất:


,title_clean,rating_avg,rating_count,genres
351,Forrest Gump,4.048011,81491.0,Comedy|Drama|Romance|War
314,"Shawshank Redemption, The",4.413576,81482.0,Crime|Drama
292,Pulp Fiction,4.188912,79672.0,Comedy|Crime|Drama|Thriller
585,"Silence of the Lambs, The",4.151342,74127.0,Crime|Horror|Thriller
2480,"Matrix, The",4.154099,72674.0,Action|Sci-Fi|Thriller
257,Star Wars: Episode IV - A New Hope,4.120189,68717.0,Action|Adventure|Sci-Fi
475,Jurassic Park,3.679175,64144.0,Action|Adventure|Sci-Fi|Thriller
522,Schindler's List,4.247579,60411.0,Drama|War
108,Braveheart,4.002273,59184.0,Action|Drama|War
2867,Fight Club,4.228311,58773.0,Action|Crime|Drama|Thriller



Movies có rating cao nhất (≥100 ratings):


,title_clean,rating_avg,rating_count,genres
45741,Planet Earth II,4.483096,1124.0,Documentary
40597,Planet Earth,4.464797,1747.0,Documentary
314,"Shawshank Redemption, The",4.413576,81482.0,Crime|Drama
45593,Band of Brothers,4.398599,1356.0,Action|Drama|War
45973,Cosmos,4.326715,277.0,(no genres listed)
840,"Godfather, The",4.324336,52498.0,Crime|Drama
49559,Blue Planet II,4.289833,659.0,Documentary
49,"Usual Suspects, The",4.284353,55366.0,Crime|Mystery|Thriller
58237,Twin Peaks,4.267361,288.0,Drama|Mystery
1190,"Godfather: Part II, The",4.261759,34188.0,Crime|Drama


In [7]:
# LƯU FILE
print("\nLưu file với features mới...")

movies.to_csv('../data/raw/movies_with_features.csv', index=False)
print("Đã lưu: data/raw/movies_with_features.csv")

# Lưu thống kê
stats = {
    'total_movies': len(movies),
    'total_ratings': len(ratings),
    'total_users': ratings['userId'].nunique(),
    'total_tags': len(tags),
    'avg_rating': ratings['rating'].mean(),
    'total_features': len(movies.columns)
}

stats_df = pd.DataFrame([stats])
stats_df.to_csv('../data/raw/dataset_stats.csv', index=False)
print("Đã lưu: data/raw/dataset_stats.csv")

print("🎉 TASK 1 HOÀN THÀNH!")


Lưu file với features mới...
Đã lưu: data/raw/movies_with_features.csv
Đã lưu: data/raw/dataset_stats.csv
🎉 TASK 1 HOÀN THÀNH!
